# Second-level analysis 2: across movies analyses

   ACTIVATE THE PY37 ENVIRONMENT BEFORE USING THIS NOTEBOOK (FOR PYMER4)

STEP4: LME of regressors across movies to find how much the slope regressor of each ROI can predict the yes vs no dichotomy

TBD: (1) where does unsure fit into this scheme? (2) Sure vs unsure LME across subs?

In [7]:
# import packages, check folders
#%reset
from pathlib import Path
import os
import numpy as np
import h5py
import pandas as pd
#import imagesc as imagesc #pip install imagesc
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import time
import pickle

%matplotlib inline

working_dir = '/Users/f0053cz/Dropbox (Dartmouth College)/postdoc_Dartmouth/HCP/fMRIScipts/code'
#working_dir = os.getcwd()
print('current directory:\n',working_dir)
path = Path(working_dir)
parent_folder = path.parent
#print('parent folder:', parent_folder)
data_file_loc = os.path.join(parent_folder,'data') # to store data we extract later in this notebook

current directory:
 /Users/f0053cz/Dropbox (Dartmouth College)/postdoc_Dartmouth/HCP/fMRIScipts/code


In [8]:
est_movie_slope = np.load(os.path.join(data_file_loc,'betapermovie_slopereg.npy'))
responses_slopereg = np.load(os.path.join(data_file_loc,'responses_slopereg.npy'))

In [9]:
from pymer4.models import Lmer

ModuleNotFoundError: No module named 'pymer4'

In [6]:
# LME for each node
nnodes = 268# 268
nmovies = 10
beta_all = np.zeros((est_movie_slope.shape[0]*nmovies,))
responses_all = np.zeros((est_movie_slope.shape[0]*nmovies,))
movies_all = np.zeros((est_movie_slope.shape[0]*nmovies,))
subs_all = np.zeros((est_movie_slope.shape[0]*nmovies,))
estResp_node = np.zeros((nnodes,))
pvalResp_node = np.zeros((nnodes,))
aicResp_node = np.zeros((nnodes,))
for n in range(nnodes):
    print('node ',n)
    for subj in np.arange(est_movie_slope.shape[0]):
        rows = np.arange(subj*nmovies,(subj+1)*nmovies)
        beta_all[rows] = est_movie_slope[subj,:,n]
        responses_all[rows] = responses_slopereg[subj,:]
        movies_all[rows] = np.arange(0,10)
        subs_all[rows] = subj

    df_node = pd.DataFrame({'Beta': beta_all,'Response': responses_all,'Movie': movies_all, 'SubjID':subs_all})
    #print('before:', len(df_node))
    df_node = df_node[(df_node['Response']!=-1)& (df_node['Response']!=9)]
    #print('after:', len(df_node))
    model = Lmer('Response ~ Beta + (1|SubjID) + (1|Movie)', data=df_node, family = "binomial") # ff: M/R, prev. trial response, rf:subjID, movie
    model.fit(summary = False, verbose = False)
    estResp_node[n] = model.coefs['Estimate'][1]
    pvalResp_node[n]= model.coefs['P-val'][1]
    aicResp_node[n] = model.AIC

node  0


NameError: name 'Lmer' is not defined

In [4]:
pd.set_option('display.max_colwidth', None) # so columns are not snipped

# loading the Shen atlas labels
shen268 = pd.read_csv(os.path.join(data_file_loc,"shen_dictionary.csv"))
shen268.head()

,1,2,3,4,5,6,7,8,9,10,...,259,260,261,262,263,264,265,266,267,268
0,"{'coords': [14.083200000000005, 56.6944, -16.684799999999996], 'name': '92.80% Frontal_Pole; 7.20% Frontal_Medial_Cortex'}","{'coords': [9.599999999999994, 17.61882352941177, -19.53411764705882], 'name': '60.56% Subcallosal_Cortex; 38.73% Frontal_Orbital_Cortex'}","{'coords': [5.270633397312864, 35.03262955854126, -17.5278310940499], 'name': '58.54% Frontal_Medial_Cortex; 34.55% Subcallosal_Cortex'}","{'coords': [15.647840531561457, 34.18604651162792, -22.544850498338867], 'name': '54.15% Frontal_Pole; 41.86% Frontal_Orbital_Cortex'}","{'coords': [8.283887468030684, 46.05882352941177, -1.6700767263427139], 'name': '47.25% Paracingulate_Gyrus; 25.03% Cingulate_Gyrus_anterior_division; 16.35% Frontal_Medial_Cortex; 10.86% Frontal_Pole'}","{'coords': [14.581460674157299, 64.75280898876406, 3.682584269662925], 'name': '100.00% Frontal_Pole'}","{'coords': [30.41914191419142, 54.79537953795381, -3.3267326732673297], 'name': '100.00% Frontal_Pole'}","{'coords': [44.469325153374236, 46.095092024539866, -4.917177914110425], 'name': '99.85% Frontal_Pole'}","{'coords': [28.814814814814817, 51.163163163163176, 18.73473473473473], 'name': '100.00% Frontal_Pole'}","{'coords': [8.521081081081078, 53.28, 23.885405405405407], 'name': '48.54% Frontal_Pole; 32.43% Paracingulate_Gyrus; 19.03% Superior_Frontal_Gyrus'}",...,"{'coords': [-10.399527186761233, 10.877068557919614, -8.056737588652481], 'name': '20.66% Subcallosal_Cortex; 19.95% Left_Putamen; 19.83% Left_Accumbens; 14.40% Left_Caudate; 13.81% no_label; 5.67% Frontal_Orbital_Cortex'}","{'coords': [-14.491582491582491, -3.54882154882155, 20.909090909090907], 'name': '55.03% Left_Caudate; 29.53% Left_Lateral_Ventrical; 11.07% Left_Thalamus'}","{'coords': [-24.77079107505071, 5.537525354969574, -0.07302231237322587], 'name': '96.96% Left_Putamen'}","{'coords': [-9.427230046948353, -25.539906103286384, -1.3755868544600958], 'name': '72.56% Left_Thalamus; 21.40% Brain-Stem; 5.58% no_label'}","{'coords': [-4.68474576271187, -10.2406779661017, 5.877966101694909], 'name': '82.71% Left_Thalamus; 13.90% Left_Lateral_Ventrical'}","{'coords': [-11.407792207792212, -25.563636363636363, 14.883116883116884], 'name': '61.82% Left_Thalamus; 31.43% Left_Lateral_Ventrical'}","{'coords': [-4.821052631578951, -21.51157894736842, -15.890526315789472], 'name': '71.16% Brain-Stem; 25.89% no_label'}","{'coords': [-4.227272727272734, -37.89772727272728, -53.26704545454545], 'name': '100.00% Brain-Stem'}","{'coords': [-6.9375, -33.13068181818181, -39.34659090909091], 'name': '99.15% Brain-Stem'}","{'coords': [-5.697885196374628, -18.966767371601208, -36.81570996978852], 'name': '100.00% Brain-Stem'}"


In [5]:
# Plotting results and picking out maximum T-scores
from nltools.data import Brain_Data
from nltools.mask import expand_mask, roi_to_brain

vidnames = ["COAXING-B", "BILLIARD-A", "DRIFTING-A", "Fishing", "Random mechanical", "Scaring", "SEDUCING-B", "STAR-A", "SURPRISING-B", "TENNIS-A"]
lbl = [str.upper(x[0:4]) for x in vidnames]

mask = Brain_Data('https://neurovault.org/media/images/8423/shen_2mm_268_parcellation.nii.gz')
mask_x = expand_mask(mask)
t = estResp_node
p = pvalResp_node
psig =  np.zeros_like(p)
psig[p <= 0.05] = 1
tsig = np.multiply(t,psig)
tsig = np.array(tsig)
img = roi_to_brain(pd.Series(tsig), mask_x)
img.plot(colorbar=True, title = 'YesvsNo')

rois = np.where(t>=0.8*np.max(t))
rois = rois[0]
roi_lbl = ["" for x in range(len(rois))]
ind = -1
for node_nr in rois:
    ind += 1
    roi_lbl[ind] = shen268[str(node_nr+1)][0]
max_rois_list = pd.DataFrame({'ROIs': rois+1, 'T-scores': t[rois], 'p-values':p[rois], 'ROINames':roi_lbl})
max_rois_list = max_rois_list.sort_values(by = ['T-scores'],ascending = False)
max_rois_list = max_rois_list.reset_index()


rois = np.where(t<=0.8*np.min(t))
rois = rois[0]
roi_lbl = ["" for x in range(len(rois))]
ind = -1
for node_nr in rois:
    ind += 1
    roi_lbl[ind] = shen268[str(node_nr+1)][0]
min_rois_list = pd.DataFrame({'ROIs': rois+1, 'T-scores': t[rois], 'p-values':p[rois], 'ROINames':roi_lbl})
min_rois_list = min_rois_list.sort_values(by = ['T-scores'],ascending = True)
min_rois_list = min_rois_list.reset_index()

  
print(max_rois_list.head())
print(min_rois_list.head())

NameError: name 'estResp_node' is not defined

In [19]:
max_rois_list.head()

,ROIs,T-scores,p-values,ROINames
0,65,0.968149,0.000001,"{'coords': [59.240374609781476, -43.72736732570239, 8.616024973985432], 'name': '51.09% Middle_Temporal_Gyrus_temporooccipital_part; 26.64% Supramarginal_Gyrus_posterior_division; 12.90% Angular_Gyrus; 6.04% Superior_Temporal_Gyrus_posterior_division'}"
1,74,0.929053,0.000117,"{'coords': [45.1231884057971, -74.23913043478261, 2.6062801932367137], 'name': '95.78% Lateral_Occipital_Cortex_inferior_division'}"
2,206,0.929788,0.000006,"{'coords': [-43.14384748700172, -70.4263431542461, -13.913344887348352], 'name': '55.19% Lateral_Occipital_Cortex_inferior_division; 29.41% Occipital_Fusiform_Gyrus; 8.65% Temporal_Occipital_Fusiform_Cortex; 6.75% Inferior_Temporal_Gyrus_temporooccipital_part'}"
3,209,1.098929,0.000009,"{'coords': [-48.17639429312581, -67.20622568093385, 0.9987029831387844], 'name': '78.86% Lateral_Occipital_Cortex_inferior_division; 15.18% Middle_Temporal_Gyrus_temporooccipital_part; 5.84% Inferior_Temporal_Gyrus_temporooccipital_part'}"


In [20]:
min_rois_list

,ROIs,T-scores,p-values,ROINames
0,120,-0.445001,0.037943,"{'coords': [21.289999999999992, -36.39, 22.400000000000006], 'name': '89.00% Right_Lateral_Ventricle; 9.50% no_label'}"
1,217,-0.538763,0.011572,"{'coords': [-23.648648648648646, -41.29054054054055, 19.858108108108112], 'name': '65.54% Left_Lateral_Ventrical; 33.78% no_label'}"
